# Inplementation of Latent Dirichlet Allocation

In [1]:
%matplotlib inline
from collections import Counter 
from scipy.special import digamma, gammaln, polygamma
import numba
from numba import jit
import numpy as np
import argparse
import cython
from collections import Counter
import nltk
nltk.download('stopwords')
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = stopwords.words('english')
countVectorizer = CountVectorizer(stop_words=stop)
from matplotlib.cm import get_cmap
from matplotlib.colors import rgb2hex
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mingjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



## 1. Abstract

Latent Dirichlet Allocation (LDA) is a very important model in machine learning area, which can automatically extract the hidden topics within a huge amount of documents and further represent the theme of each document as an ensemble of topics. It is a three-level hierarchical Bayesian model, in which each
item of a collection is modeled as a finite mixture over an underlying set of topics. Each topic is, in
turn, modeled as an infinite mixture over an underlying set of topic probabilities. In the context of
text modeling, the topic probabilities provide an explicit representation of a document. Due to the intractability of LDA posterior inference, many approximate inference techniques have been invented.In this study, I present
efficient approximate inference techniques based on variational methods and an EM algorithm as well as a gibbs algorithm for
empirical Bayes parameter estimation.

Keywords: Latent Dirichlet Allocation, topic models, text mining, Gibbs sampling, variational inference

## 2. Background


### 2.1 Topic modeling

Knowing what people are talking about and understanding their problems and opinions is highly valuable to businesses, administrators, political campaigns. And it’s really hard to manually read through such large volumes and compile the topics. With the plethora amount of information available on the Internet, the topic of knowledge management has become ever so more important. In text mining, we often have collections of documents, such as blog posts or news articles, that we’d like to divide into natural groups so that we can understand them separately. Topic modeling is the process of identifying topics in a set of documents. It is a method for unsupervised classification of such documents, similar to clustering on numeric data, which finds natural groups of items even when we’re not sure what we’re looking for. Topic Modeling automatically discover the hidden themes from given documents, and it can be useful for search engines, customer service automation, and any other instance where knowing the topics of documents is important. There are multiple methods of topic modeling, Latent Dirichlet Allocation (LDA) is one of the most popular algorithms.


### 2.2 Latent Dirichlet Allocation


Latent Dirichlet Allocation (LDA) can be used for a variety of tasks, from clustering customers based on product purchases to automatic harmonic analysis in music. However, it is most commonly associated with topic modeling in text mining. It is a particularly popular method for fitting a topic model with a collection of text documents. LDA is a "bag-of-words" model, which means that the order of words does not matter. LDA supposes that there is some fixed vocabulary (composed of $V$ distinct terms) and $K$ different topics. It treats each document as a mixture of topics, and each topic as a mixture of words. Observations are referred to as documents. The feature set is referred to as vocabulary. A feature is referred to as a word. And the resulting categories are referred to as topics. This allows documents to “overlap” each other in terms of content, rather than being separated into discrete groups, in a way that mirrors typical use of natural language. As an unsupervised learning algorithm, LDA attempts to describe a set of observations as a mixture of different categories. These categories are themselves a probability distribution over the features. LDA is a generative probability model, which means it attempts to provide a model for the distribution of outputs and inputs based on latent variables. This is opposed to discriminative models, which attempt to learn how inputs map to outputs.



## 3. Description of algorithm


LDA is a probabilistic model of text. It casts the problem of discovering
themes in large document collections as a posterior inference problem. It allows us to visualize the hidden thematic structure in large collections, and
generalize new data to fit into that structure. The model builds on latent semantic analysis, and it relates to PCA and matrix factorization and it was independently invented for genetics.

With the vocabulary and topics chosen, the LDA model assumes that we have a
set of $M$ documents (each “document” may be a paragraph or other section of the
text, rather than a “full” document). The m-th document consists of $N_m$ words, and
a probability distribution $\theta_m$ over the topics is drawn from a Dirichlet distribution
with parameter $\alpha$. Thus $\theta_{m,k}$ is the probability that document $m$ is assigned the
label $k$. 

The Dirichlet distribution is an exponential family distribution that is conjugate to the multinomial. Given a multinomial observation, the posterior distribution of $\theta$ is a Dirichlet. The parameter $\alpha$ controls the mean shape and sparsity of $\theta$. The topic proportions are a $K$ dimensional Dirichlet.
The topics are a $V$ dimensional Dirichlet.

LDA assumes the following generative process for each document m in a corpus:

1.Choose $N \sim$ Poisson($\epsilon$). 


2.Choose $\theta \sim Dir(\alpha)$. 


3.For each of the $N$ word $w_n$: 

   (a) Choose a topic $z_n \sim$ Multinomial$(\theta)$. 

   (b) Choose a word $w_n$ from $p(w_n|z_n,\beta)$, a multinomial probability conditioned on the topic $z_n$. 

Several simplifying assumptions are made in this basic model.

First, the dimensionality $k$ of the Dirichlet distribution (and thus the dimensionality
of the topic variable $z$) is assumed known and fixed. Second, the word probabilities are parameterized
by a $k x V $ matrix $\beta$ where $\beta_{ij} = p(w^j = 1| z^i = 1)$, which for now we treat as a fixed quantity
that is to be estimated. Finally, the Poisson assumption is not critical to anything that follows and
more realistic document length distributions can be used as needed. Furthermore, note that $N$ is
independent of all the other data generating variables ($\theta$ and $z$). It is thus an ancillary variable and
we will generally ignore its randomness in the subsequent development.

Given the parameters $\alpha$ and $\beta$, the joint distribution of a topic mixture $\theta$, a set of $N$ topics $z$, and a set of $N$ words $w$ is given by:

$$
p(\theta, z, w|\alpha, \beta)=p(\theta|\alpha)\prod _{ n=1 }^{ N }{ p(z_n|\theta)p(w_n|z_n,\beta) } 
$$
where $p(z_n|\theta)$ is simply $\theta_i$ for the unique i such that ${ z }_{ n }^{ i }=1$. Integrating over $\theta$ and summing over z, we obtain the marginal distribution of a document:

$$
p(w|\alpha, \beta) = \int { p(\theta |\alpha )(\prod _{ n=1 }^{ N }{ \sum { p({ z }_{ n }|\theta )p({ w }_{ n }|{ z }_{ n },\beta ) }  } )d\theta  } 
$$Finally, taking the product of the marginal probabilities of single documents, we obtain the probability of a corpus:

$$
p(D|\alpha, \beta) = \prod _{ d=1 }^{ M }{ \int { p(\theta_d |\alpha )(\prod _{ n=1 }^{ N_d }{ \sum { p({ z }_{ dn }|\theta )p({ w }_{ dn }|{ z }_{ dn },\beta ) }  } )d\theta_d  }  } 
$$


## 4. Optimization of model performance
In this section, I first implement the algorithm using plain Python in a straightforward way from the description of the algorihtm. Then I used three methods for optimization, including Variational EM, TimeIt, JIT, and Cython

###  4.1 Implement the algorithm using plain Python - Gibbs Sampling

###  Gibbs sampler process

To infer the latent parameters is a problem of Bayesian inference. Therefore, we could use Gibbs Sampling to estimate latent parameters. Gibbs sampling is an MCMC sampling method in which we construct a Markov chain which is used to sample from a desired joint (conditional) distribution
$$ \mathbb{P}(x_1, · · · , x_n|y)$$

Often it is difficult to sample from this high-dimensional joint distribution, while it
may be easy to sample from the one-dimensional conditional distributions
$$ \mathbb{P}(x_i|x_{-i}, y)$$
where $x_{-i} = x_1,..., x_{i-1}, x_{i+1},..., x_n$.

Our results highlight the importance of averaging over multiple samples in
LDA parameter estimation, which is the benefit of gibbs sampling.

#### Gibbs in LDA

Given $i={1,...,N_{D}}$ the document index, $v = {1,...,N_{w}}$ the word index, $k = {1,...,N_K}$ the topic index, LDA assuems:

$\pi_{i} \sim Dir(\pi_{i} | \alpha)$


$z_{iv} \sim Cat(z_{iv} | \pi_{i})$


$\beta_k \sim Dir(\beta_k | \gamma)$


$y_{iv} \sim Cat(y_{iv} | z_{iv} = k,\beta)$

Here we see gamma and alpha as parameters from the assumption of Dirichlet priors, both indicating how narrow or spread the document topic and topic word distribution $b_k$ are. We could see the posteriors of $\theta_i$ and $\beta_i$ also follow the Dirichlet distribution. Their posterior means are:

$$
\theta_{i,k} = \frac { { n }_{ i }^{ k }+{ \alpha  }_{ k } }{ \sum _{ k=1 }^{ K }{ { n }_{ i }^{ k }+{ \alpha  }_{ k } }  } 
$$$$
\beta_{k,w} = \frac { { n }_{ w }^{ k }+{ \beta  }_{ w } }{ \sum _{ w=1 }^{ W }{ { n }_{ w }^{ k }+{ \beta  }_{ w } }  } 
$$

I follow the following steps for sampling:

1) Assume each document generated by selecting the topic first. Thus, sample $\pi_{i}$, the topic distribution for ith document.

2) Assume each words in ith document comes from one of the topics. Therefore, we sample $Z_{iv}$, the topic for each word v in document i.

3) Assume each topic is composed of words, sample $\beta_k$ , the distribution those words for particular topic $k$.

4) Finally, to actually generate the word, given that we already know it comes from topic k, we sample the word $w_{iv}$ given the k-th topic word distribution.




I will infer those variables using Gibbs Sampling algorithm. In short, it works by sampling each of those variables given the other variables (full conditional distribution). Because of the conjugacy, the full conditionals are as follows:
$$ p(z_{iv} = k | \pi_i, \beta_k) \propto exp(log \pi_{ik} + log \beta_{k,w_{iv}})$$


$$ p (\beta_k | z_{iv} = k, \pi_i) = Dir(\gamma + \sum_i\sum_l I (y_{il} = v, z_{il} = k))$$

$$p (\pi_i | z_{iv} = k, \beta_k) = Dir(\alpha + \sum_l I (z_{il} = k))
$$


### Python code for Gibbs sampling
### Part 1: preprocessing data
As pre-processing, we first need to spot each word's position in the dictionary. Then we need to generate some basic count matrices, including a word-topic count matrix, word-topic assignment matrix and document-topic count matrix. We also need to give initial values to parameters of alpha,gamma, Pi, B

In [35]:
def pos_in_voc(corpus, voc):
    '''generate a matrix to store each word's position in the dictionary '''
    loc_doc=np.copy(corpus)
    for i in range(len(corpus)):
        for j in range(len(corpus[i])):
            idx=np.where(voc==corpus[i][j])[0]
            if len(idx)==0:
                voc=np.append(voc,corpus[i][j]) 
                idx=np.where(voc==corpus[i][j])[0]
            loc_doc[i][j]=idx[0]
    return loc_doc, voc

def pre_processing(corpus,voc, N_K):
    # wt: a word-topic count matrix
    # Z: word-topic assignment matrix
    # dt: document-topic count matrix
    # n_z : the total number of words assigned to topic z
    # n_m: the total number of words in each document in corpus
    Z = []
    V=len(voc)  #number of words in vocabulary
    dt = np.zeros((len(corpus), N_K))
    wt = np.zeros((N_K, V))
    n_z = np.zeros(N_K)
    n_m = []
    # randomly assign topics to words in each doc
    for m in range(len(corpus)):
        z_n = []
        doc=corpus[m]
        for n in range(len(doc)):
            t=doc[n]
            z = np.random.randint(0, N_K)
            z_n.append(z)
            dt[m, z] += 1
            wt[z, t] += 1

            n_z[z] += 1
        Z.append(np.array(z_n))
    
    for i in range(len(corpus)):
        n_m.append(len(corpus[i]))
        
    return Z, dt, wt, n_z, np.array(n_m), V

def initialize(N_D, N_W, N_K, doc):
    """Initialize the parameters"""
    # Dirichlet priors
    alpha = 1
    gamma = 1

    # Z := word topic assignment
    Z = np.zeros(shape=[N_D, N_W])

    for i in range(N_D):
        for l in range(N_W):
            Z[i, l] = np.random.randint(N_K)  # randomly assign word's topic

    # Pi(theta) := document topic distribution
    Pi = np.zeros([N_D, N_K])

    for i in range(N_D):
        Pi[i] = np.random.dirichlet(alpha*np.ones(N_K))

    # B(beta) := word topic distribution
    B = np.zeros([N_K, N_W])

    for k in range(N_K):
        B[k] = np.random.dirichlet(gamma*np.ones(N_W))
    
    return alpha,gamma, Pi, B

### Part 2: Inplement equations to simulate parameters

Based on the original paper, I inplement the equations for calculating the posterior for theta and beta with two functions as below. And I wrote another function to realize the sampling process.

In [36]:
def resample_theta(N_D, n_m, N_K,dt, Pi):
    """Update the theta value"""
    theta = (dt + Pi)/(n_m[:, None] + N_K * Pi)
    
    return theta

def resample_beta( wt, n_z,lenV, alpha):
    """Update the beta value"""
    beta = (wt + alpha)/(n_z[:,None] + lenV *alpha)
    return beta

In [37]:
def sample_conditionally(loc_doc,doc,doc_word, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt):
    z0=Z[doc][doc_word]  #initial topic assignment to word
    pos=loc_doc[doc][doc_word] # position of the word in voc
    
    #exclude the word
    dt[doc][z0]=dt[doc][z0]-1
    wt[z0,pos]=wt[z0,pos]-1
    
    # resample topic for current word
    p_z = ((wt[:,pos] + phi)/(n_z + lenV * phi)) * ((dt[doc,:] + alpha)/(n_m[doc] + N_K * alpha))

    p_z=p_z/p_z.sum()
    z1 = np.random.multinomial(1, p_z).argmax()
    Z[doc][doc_word]= z1
    
    # add the word back
    dt[doc][z1]=dt[doc][z1]+1
    wt[z1,pos]=wt[z1,pos]+1
    
    return Z, dt, wt

### Part 3: Inplement Gibbs sampling process
I wrote a main function lda_gibbs to call the three functions in part 2 and realize the Gibbs sampling process. 

In [38]:
def lda_gibbs(loc_doc,iteration, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi):
    """Gibbs Sampler in LDA"""
    np.random.seed(116)
    beta_sum=[]
    theta_sum=[]

    for i in range(iteration):
        for j in range(len(loc_doc)):
            for k in range(len(loc_doc[j])):
                Z_new, dt, wt = sample_conditionally(loc_doc,j,k, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt)

        
        temp_beta=resample_beta( wt, n_z,lenV, alpha)
        beta_sum.append(temp_beta)
        
        temp_theta = resample_theta(N_D, n_m, N_K,dt, Pi)
        theta_sum.append(temp_theta)
        
    return beta_sum,   theta_sum, wt


### 4.2 Optimization 1 : EM Algorithm

The goal of LDA is to compute the 2 hidden parameters often commonly denoted as $\theta$ and $\phi$, where $\theta$ represents topic-document distribution and $\phi$ represents the word-topic distribution. Variational inference seems to be the preferred method for getting fast and accurate results in most software implementations.

LDA model aims to uncover latent topics in documents of text, so it is a latent variable model and can be optimized with variational expectation maximization algorithm (EM). Suppose we have a model with parameter $\theta$ and latent variables Z, EM is a mechanism for computing the values of $\theta$ that maximize the likelihood based on some observed data X.



Given the parameters $\alpha$ and $\beta$, the joint distribution of a topic mixture $\theta$, a set of $N$ topics $z$, and a set of $N$ words $w$ is given by:

$$
p(\theta, z, w|\alpha, \beta)=p(\theta|\alpha)\prod _{ n=1 }^{ N }{ p(z_n|\theta)p(w_n|z_n,\beta) } 
$$
The (problematic) log-evidence of a single document:
$$log p(w | \alpha,\beta) = log \int p(\theta | \alpha) \prod _{n=1}^{N}\sum_{z_n} p(z_n | \theta)p(w_n | z_n,\beta)d\theta$$
Deriving the ELBO:

Given the summation over Z, introducing some distribution q(z) would provide the expection, where q(z) is some didstribution over Z with parameters $\lambda$ and known form (e.g. Gaussian). It's often referred to as a variational distribution.
$$ \mathbb{E}_{q(z)} [ log( p(\theta | \alpha) / q(z) \prod_{n=1}^{N} p (z_n | \theta) p(w_n | z_n,\beta))]$$
KL term:
$$ KL(q(Z)|| \frac{p(\theta,z,w|\alpha,\beta)}{p(w|\alpha,\beta)})$$

Peering at the denominator, we see that the expression under the integral is exponential in the number of words $N$; for any non-trivial $N$ and number of topics, it is intractable to compute. As such, the "ideal" E-step solution $q(Z)=p(\theta,z|w,\alpha,\beta)$ admits no analytical form.

The EM algorithm is a repeated alternation between E-step and M-step.

E-step:

The aim of variational inference is to learn the values of variational paramters. With the learnt variational parameters, we can evaluate the posterior probabilities of hidden variables. In this step, the q distribution is set equal to the posterior distribution for the current parameter variable $\theta^{old}$, causing the lower bound to move up to the same value as the log likelihood function, with the KL divergence vanishing. Particularly in the LDA model, we perform variational inference for learning variational paramters and calcuate the variational paramters Phi and gamma.


In [14]:
def E_step(Phi, gamma, alpha, Beta, corpus, voc, k, M):
    '''E-step: variational inference'''
    likelihood = 0.0
    
    for d in range(0,M):
        words = np.array(corpus[d])
        N = len(words)
        phi = Phi[d]
        
        conv_counter = 0
        #
        while conv_counter < 100:
            
            phi_old = phi
            phi = np.zeros([N,k])
            gamma_old = gamma[d, :]
            
            for n in range(0,N):
                word = words[n]
                w_in_voc =np.where(voc == word)
                if len(w_in_voc[0]) > 0: # word exists in vocabulary
                    phi[n,:] = Beta[:,w_in_voc[0][0]]* np.exp(digamma(gamma[d,:]) - digamma(np.sum(gamma[d,:])))
                    phi[n,:] = phi[n,:] / np.sum(phi[n,:])   
            alpha = np.ones([M,k])
            gamma[d, :] = alpha[d, :] + np.sum(phi, axis=0)    
            
            conv_counter += 1
            # Check if gamma and phi converged
            if np.linalg.norm(phi - phi_old) < 1e-3 and np.linalg.norm(gamma[d,:] - gamma_old) < 1e-3:
                Phi[d] = phi               
                likelihood += compute_likelihood(Phi[d], gamma[d,:], alpha[d,:], Beta, corpus[d], voc, k)
                conv_counter=100
                
    return Phi, gamma, likelihood

M-step:

In this step, we need to estimate the model parameters by using the variational lower bound as a surrogate for the marginal log likelihood, with the variational paraters. The distribution q(Z) is held fixed and the lower bound is maximized with respecct to the parameter vector $\theta$ to give a revised value $\theta^{new}$. Since the KL divergence is nonnegative, this result in an increase in the log likelihood at least as much as the lower bound does. Particularly in the LDA model, we perform parameter estimation to update alpha and beta.

In [15]:
def M_step(Phi, gamma, alpha, corpus, voc, k, M):
    V = len(voc)
    # 1 update Beta
    Beta = np.zeros([k,V])
    for d in range(0,M):
        words = np.array(corpus[d])
        voc_pos = np.array(list(map(lambda x: np.in1d(words, x),voc)))
        Beta += np.dot(voc_pos, Phi[d]).transpose()
    Beta = Beta / Beta.sum(axis=1).reshape(k,1)

    # 2 update alpha
    for i in range(1000):
        old_alpha = alpha
    # Calculate the gradient, Hessian 
        g = M*(digamma(np.sum(alpha))-digamma(alpha)) + np.sum(digamma(gamma)-np.tile(digamma(np.sum(gamma,axis=1)),(k,1)).T,axis=0)
        h = -M * polygamma(1,alpha)
        z = M * polygamma(1,np.sum(alpha))
        c = np.sum(g/h)/(1/z+np.sum(1/h))
    # Update alpha
        alpha -= (g-c) / h
        if np.sqrt(np.mean(np.square(alpha-old_alpha)))<1e-4:
            break

    return alpha, Beta

### 4.3 Optimization 2 :  Timeit and Just In Time Compilation

#### To test the algrithm, I created a function to generate a test dataset with known parameters for testing purpose

In [10]:
def test_data(docnum = 200):
    """Generating test data."""
    def generating(words, probs):
        '''generate words based on given probabilities'''
        wordslist = []; all_theta = []
        c = (0.4,0.5)
        theta = np.random.dirichlet(c)
        all_theta.append(theta)
        N = np.random.poisson(15)  
        for i in range(0,N):
            # Draw topics from multinomial distribution
            z_n = np.random.multinomial(1,theta)
            z_n = np.nonzero(z_n)[0][0]
            w_n = np.random.multinomial(1,probability[z_n,:])
            w_n = np.nonzero(w_n)[0][0]
            wordslist.append(words[z_n][w_n])
        return wordslist, all_theta
    # generate vocabulary
    voc = ['bayesian','probability','posterior','prior',   #first topic
            'add','minus','multiply','divide']          #second topic
    #Initializing words and topics
    words = []
    words.append(['bayesian','probability','posterior','prior'])
    words.append(['add','minus','multiply','divide'])
    probability = np.array(([0.25,0.25,0.25,0.25],  #first topic has equal paobablities
                           [0.7,0.05,0.05,0.2]))    #second topic has unequal paobablities
    
    docs = []; doci = []
    for i in range(0, docnum):
    	doci,theta = generating(words, probability)
    	docs.append(doci)
    return docs, voc

In [11]:
documents, vocabulary=test_data(20)
vocabulary=np.array(vocabulary)
loc_doc, vocabulary=pos_in_voc(documents, vocabulary)

phi=0.2
N_D = len(documents)  
N_K = 2  
max_iter=5000

Z, dt, wt, n_z, n_m, lenV=pre_processing(loc_doc,vocabulary, N_K)
alpha,gamma, Pi, B=initialize(N_D, lenV, N_K, loc_doc)
beta_sum,  theta_sum, wtt=lda_gibbs(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)    


In [17]:
def resample_theta(N_D, n_m, N_K,dt, Pi):
    """Update the theta value"""
    theta = (dt + Pi)/(n_m[:, None] + N_K * Pi)
    
    return theta

In [20]:
@jit
def resample_theta_jit(N_D, n_m, N_K,dt, Pi):
    """Update the theta value"""
    theta = (dt + Pi)/(n_m[:, None] + N_K * Pi)
    
    return theta

In [19]:
%timeit resample_theta(N_D, n_m, N_K,dt, Pi)

14.3 µs ± 4.86 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
%timeit resample_theta_jit(N_D, n_m, N_K,dt, Pi)

10.2 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
def lda_gibbs(loc_doc,iteration, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi):
    """Gibbs Sampler in LDA"""
    np.random.seed(116)
    beta_sum=[]
    theta_sum=[]

    for i in range(iteration):
        for j in range(len(loc_doc)):
            for k in range(len(loc_doc[j])):
                Z_new, dt, wt = sample_conditionally(loc_doc,j,k, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt)

        
        temp_beta=resample_beta( wt, n_z,lenV, alpha)
        beta_sum.append(temp_beta)
        
        temp_theta = resample_theta(N_D, n_m, N_K,dt, Pi)
        theta_sum.append(temp_theta)
        
    return beta_sum,   theta_sum, wt


In [25]:
@jit
def lda_gibbs_jit(loc_doc,iteration, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi):
    """Gibbs Sampler in LDA"""
    np.random.seed(116)
    beta_sum=[]
    theta_sum=[]

    for i in range(iteration):
        for j in range(len(loc_doc)):
            for k in range(len(loc_doc[j])):
                Z_new, dt, wt = sample_conditionally(loc_doc,j,k, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt)

        
        temp_beta=resample_beta( wt, n_z,lenV, alpha)
        beta_sum.append(temp_beta)
        
        temp_theta = resample_theta(N_D, n_m, N_K,dt, Pi)
        theta_sum.append(temp_theta)
        
    return beta_sum,   theta_sum, wt


In [26]:
max_iter=50
phi=0.5

In [35]:
%timeit lda_gibbs(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)

51 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit lda_gibbs_jit(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)

27 ms ± 990 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### By implying Just In Time Compilation, around 24ms are saved in the main algorithm.

### 4.4 Optimization 3 : Cython

In [2]:
%load_ext cython

In [3]:
%%cython -a

import numpy as np
import cython
@cython.boundscheck(False)
@cython.wraparound(False)

def lda_gibbs_cy(loc_doc,iteration,N_K, lenV, n_z, n_m,gamma, alpha, phi, Z , dt, wt, N_D, B, Pi):
    cdef int i, n, w, t
    
    
    np.random.seed(116)
    beta_sum=[]
    theta_sum=[]
    def pos_in_voc(corpus, voc):
        loc_doc=np.copy(corpus)
        for i in range(len(corpus)):
            for j in range(len(corpus[i])):
                idx=np.where(voc==corpus[i][j])[0]
                if len(idx)==0:
                    voc=np.append(voc,corpus[i][j]) 
                    idx=np.where(voc==corpus[i][j])[0]
                loc_doc[i][j]=idx[0]
        return loc_doc, voc

    def sample_conditionally(loc_doc,doc,doc_word, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt):
        z0=Z[doc][doc_word]  #initial topic assignment to word
        pos=loc_doc[doc][doc_word] # position of the word in voc
    
        #exclude the word
        dt[doc][z0]=dt[doc][z0]-1
        wt[z0,pos]=wt[z0,pos]-1
    
        # resample topic for current word
        p_z = ((wt[:,pos] + phi)/(n_z + lenV * phi)) * ((dt[doc,:] + alpha)/(n_m[doc] + N_K * alpha))
        p_z=p_z/p_z.sum()
        z1 = np.random.multinomial(1, p_z).argmax()
        Z[doc][doc_word]= z1
    
        # add the word back
        dt[doc][z1]=dt[doc][z1]+1
        wt[z1,pos]=wt[z1,pos]+1
    
        return Z, dt, wt
    
    def resample_beta(wt, n_z,lenV, alpha):
        beta = (wt + alpha)/(n_z[:,None] + lenV *alpha)
        return beta

    def resample_theta(N_D, n_m, N_K,dt, Pi):
        theta = (dt + Pi)/(n_m[:, None] + N_K * Pi)
        return theta

    for i in range(iteration):
        for j in range(len(loc_doc)):
            for k in range(len(loc_doc[j])):
                Z_new, dt, wt = sample_conditionally(loc_doc,j,k, N_K, lenV, n_z, n_m, alpha, phi, Z, dt, wt)

        temp_beta=resample_beta( wt, n_z,lenV, alpha)
        beta_sum.append(temp_beta)
        temp_theta = resample_theta(N_D, n_m, N_K,dt, Pi)
        theta_sum.append(temp_theta)
    return beta_sum,   theta_sum, wt

In [12]:
%timeit lda_gibbs(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)

4.91 s ± 1.32 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit lda_gibbs_cy(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)

2.8 s ± 79.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### We are improving by 2s using cython. 

## 5. Applications to test data

In this section I use both EM and Gibbs sampler to test the algorithm based on the test data we generated. We can see that the   top words for the two topics given by EM match the generated test data, indicating a successfully model inplementation. And Gibbs sampler calculates the parameters for the dataset.

### 5.1 EM method

In [22]:
def initialize_parameters(corpus, voc, k, M):
    '''give initial values to parameters'''
    Phi = []
    gamma = np.zeros([M,k])
    alpha = np.ones([M,k])

    for m in range(0,M):
        doc = np.array(corpus[m])
        N = len(doc)
        phi = np.ones([N,k])/k
        gamma[m,:] = alpha [m,:] + N/k
        Phi.append(phi)
    # Initialize Beta
    Beta = np.random.uniform(0,1,(k,len(voc)))
    Beta = Beta/Beta.sum(axis=1).reshape(k,1)
    return Phi, gamma, alpha, Beta


def compute_likelihood(Phi, gamma, alpha, Beta, doc, voc, k):
    '''calculate the likelihood'''
    likelihood = 0.0
    V = len(voc)
    words = np.array(doc)
    N = len(words)
    
    alpha_sum = 0.0
    phi_gamma_sum = 0.0
    phi_lgb_sum = 0.0
    e_sum = 0.0
    gamma_sum = 0.0
    
    alpha_sum += gammaln(alpha.sum())  
    gamma_sum -= gammaln(gamma.sum()) 
    for i in range(0,k):
        alpha_sum -= gammaln(alpha[i]) + (alpha[i] - 1) * (digamma(gamma[i]) - digamma(gamma.sum()))
        Phi_p= Phi[:,i] > 0
        w_ind = np.array(list(map(lambda x: np.sum(np.in1d(voc, x)),words[Phi_p])))   
        phi_gamma_sum = np.sum(Phi[Phi_p,i] * (digamma(gamma[i]) - digamma(gamma.sum())))
        e_sum = np.dot(Phi[Phi_p,i],np.log(Phi[Phi_p,i]))
        b_p=Beta[i,:]>0
        phi_lgb_sum += np.sum(np.outer((Phi[Phi_p,i] * w_ind), np.log(Beta[i,b_p])))
        gamma_sum += gammaln(gamma[i]) - (gamma[i] - 1) * (digamma(gamma[i]) - digamma(gamma.sum()))
    
    likelihood += (alpha_sum + phi_gamma_sum + phi_lgb_sum - gamma_sum - e_sum) 
    return likelihood

    
def variational_EM(Phi_init, gamma_init, alpha_init, Beta_init, corpus, voc, k, M):
    '''EM inplementation'''
    print('Variational EM')
    likelihood = 0
    likelihood_old = 0
    iteration = 1 # Initialization step is the first step
    Phi = Phi_init
    gamma = gamma_init
    alpha = alpha_init
    Beta = Beta_init
    while iteration <= 100 and (iteration <= 2 or np.abs((likelihood-likelihood_old)/likelihood_old) > 1e-4):
        # Update parameters 
        likelihood_old = likelihood
        Phi_old = Phi 
        gamma_old = gamma 
        alpha_old = alpha
        Beta_old = Beta
    
        Phi, gamma, likelihood = E_step(Phi_old, gamma_old, alpha_old, Beta_old, corpus, voc, k, M)
        alpha, Beta = M_step(Phi, gamma, alpha_old, corpus, voc, k, M)
        iteration += 1
            
    return Phi, gamma, alpha, Beta, likelihood
    
def inference_method(corpus, voc, k=2):
    '''use EM to do LDA'''
    M = len(corpus)   # nbr of documents
    Phi_init, gamma_init, alpha_init, Beta_init = initialize_parameters(corpus, voc, k, M)
    Phi, gamma, alpha, Beta, likelihood = variational_EM(Phi_init, gamma_init, alpha_init, Beta_init, corpus, voc, k, M)
    
    return Phi, gamma, alpha, Beta, likelihood

In [23]:
vocabulary=np.array(vocabulary)

Phi, gamma, alpha, Beta, likelihood = inference_method(documents, vocabulary)
topk=2 #get top 2 key words
N_K = 2
for i in range(N_K):
	print("Top key words for topic",i+1)
	sorted_idx=np.argsort(-Beta[i], axis=0)

	for j in range(topk):
		aa=sorted_idx[j]
		print(vocabulary[aa])

Variational EM
Top key words for topic 1
add
divide
Top key words for topic 2
probability
prior


### 5.2 Gibbs Sampler

In [20]:
max_iter=5000
Beta=beta_sum[0]
Theta=theta_sum[0]
### print Beta
for i in range(1, max_iter):
	for j in range(N_K):
		Beta[j]= [Beta[j][q] + beta_sum[i][j][q] for q in range(len(vocabulary))]

for i in range(len(Beta)):
	Beta[i]=Beta[i]/max_iter
print("Beta\n",Beta)

### print Theta
for i in range(1,max_iter):
	for j in range(len(Theta)):
		Theta[j]=Theta[j]+theta_sum[i][j]

for i in range(len(Theta)):
	Theta[i]=Theta[i]/max_iter
print("Theta\n",Theta)

Beta
 [[0.1257485  0.09580838 0.13772455 0.20359281 0.2994012  0.03592814
  0.02994012 0.08383234]
 [0.14201183 0.18934911 0.10650888 0.14201183 0.27218935 0.02366864
  0.01775148 0.09467456]]
Theta
 [[0.29614981 0.69629255]
 [0.19932028 0.78317823]
 [0.54718062 0.4519343 ]
 [0.63531918 0.38385685]
 [0.75078107 0.25077622]
 [0.34031563 0.65296025]
 [0.45291877 0.54816929]
 [0.39636953 0.60694812]
 [0.33700359 0.65310273]
 [0.43298647 0.56939156]
 [0.61030506 0.39933797]
 [0.5        0.5       ]
 [0.59318701 0.40568016]
 [0.41966477 0.57966755]
 [0.60822742 0.39392018]
 [0.5        0.5       ]
 [0.67843489 0.32129182]
 [0.55677014 0.43896897]
 [0.67336674 0.3203124 ]
 [0.68958403 0.32533648]]


## 6. Applications to real data sets

####  After testing the model, we could apply our LDA models to the real datasets. 
#### Dataset 1: 
I tested the algorithm on the real-world examples from the orignal paper. The dataset is based on the 2204 documents from the Associated Press. The original dataset is available at: https://github.com/blei-lab/lda-c/tree/master/example
Note: Since it may take hours to run the whole dataset, here only use 6 documents from the original dataset as a demonstration

#### Dataset 2: 
I used another real dataset not in the original paper. This data set contains several public files of religious and spiritual texts. Also included is a “wildcard” file on the subject of machine super intelligence. This file is licensed under a Creative Commons Attribution-Share Alike 2.5 Switzerland License. More information can be found at: https://creativecommons.org/licenses/by-sa/2.5/ch/deed.en. The original dataset is available at:https://www.kaggle.com/metron/public-files-of-religious-and-spiritual-texts

In [3]:
#####    pre-processing documents
def get_doc(file_name,stopwords_file):
    '''preprocess the real dataset to tokenize the words'''
    texts = []
    special_chars = '!"#$£@%&/()=?.,+-*\':;_´`1234567890'
    with open(file_name, 'r') as infile:
        copy = False
        text = ''
        for line in infile:
            if copy:
                if line.strip() == '</TEXT>':
                    text = text.lower()
                    texts.append(text)
                    text = ''
                    copy = False
                else:
                    for char in special_chars:
                        line = line.replace(char, '')
                    text += line
            else:
                if line.strip() == '<TEXT>':
                    copy = True
        tmp_texts = np.array(texts)
    
    stop_words_line = []
    with open(stopwords_file, 'r') as infile:
        data=infile.read().replace(',', ' ')
        for word in data.split():
            stop_words_line.append(word)
        
    stop_words = np.array(stop_words_line)
        
    corpus = []
    for text in tmp_texts:
        words = np.array(text.split())
            
        stopwords_filtered_document = [w for w in words if w not in stop_words]
        single_words = [k for k, v in Counter(stopwords_filtered_document).items() if v == 1 ]
        final_filtered_document = [w for w in stopwords_filtered_document if w not in single_words]
            
        if not final_filtered_document: # Document is empty, Shape = []
            continue
        corpus.append(final_filtered_document)
    return corpus

### 6.1 EM method

#### Dataset 1

In [95]:
documents = get_doc('data/sample1.txt',  'data/stopwords.txt')
vocabulary=np.genfromtxt('data/vocab.txt',  dtype='str')
N_K = 3
Phi, gamma, alpha, Beta, likelihood = inference_method(documents, vocabulary,N_K)

topk=3 #get top 3 key words
for i in range(N_K):
	print("Top key words for topic",i+1)
	sorted_idx=np.argsort(-Beta[i], axis=0)

	for j in range(topk):
		aa=sorted_idx[j]
		print(vocabulary[aa])

Variational EM
Top key words for topic 1
peres
offer
official
Top key words for topic 2
police
mrs
liberace
Top key words for topic 3
noriega
panama
president


#### Dataset 2 

In [23]:
documents = get_doc('data/real_data.txt',  'data/stopwords.txt')
vocabulary=np.genfromtxt('data/vocab.txt',  dtype='str')
N_K = 3
Phi, gamma, alpha, Beta, likelihood = inference_method(documents, vocabulary,N_K)

topk=3 #get top 3 key words
for i in range(N_K):
	print("Top key words for topic",i+1)
	sorted_idx=np.argsort(-Beta[i], axis=0)

	for j in range(topk):
		aa=sorted_idx[j]
		print(vocabulary[aa])

Variational EM
Top key words for topic 1
god
lord
signs
Top key words for topic 2
lord
god
mercy
Top key words for topic 3
god
mine
grace


### 6.2 Gibbs sampler

#### Dataset 1

In [39]:
documents = get_doc('data/sample1.txt',  'data/stopwords.txt')
loc_doc, vocabulary=pos_in_voc(documents, vocabulary)   
phi=0.2
N_D = len(documents) 

N_K = 3  
max_iter=1000 

Z, dt, wt, n_z, n_m, lenV=pre_processing(loc_doc,vocabulary, N_K)

alpha,gamma, Pi, B=initialize(N_D, lenV, N_K, loc_doc)
beta_sum,  theta_sum, wtt=lda_gibbs(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)    

Beta=beta_sum[0]
Theta=theta_sum[0]

Beta=np.array(beta_sum).mean(axis=0)
print("Beta\n",Beta)

### print Theta
for i in range(1,max_iter):
	for j in range(len(Theta)):
		Theta[j]=Theta[j]+theta_sum[i][j]

for i in range(len(Theta)):
	Theta[i]=Theta[i]/max_iter
print("Theta\n",Theta)

Beta
 [[9.32053313e-05 9.36713580e-05 9.32053313e-05 ... 9.38577687e-05
  9.35781527e-05 9.32985367e-05]
 [9.28677563e-05 9.78826152e-05 9.28677563e-05 ... 9.29606241e-05
  9.29606241e-05 9.30534918e-05]
 [9.33096949e-05 2.74423813e-04 9.33096949e-05 ... 2.79182607e-04
  2.79462536e-04 2.79649156e-04]]
Theta
 [[0.00732736 0.98900188 0.00695349]
 [0.0033036  0.98979103 0.00456677]
 [0.00112588 0.98394885 0.00271967]
 [0.00816111 0.02875608 0.94995466]
 [0.09381157 0.01934802 0.8735147 ]
 [0.0021055  0.00194502 0.99038158]]


#### Dataset 2

In [40]:
documents = get_doc('data/real_data.txt',  'data/stopwords.txt')

vocabulary=np.genfromtxt('data/vocab.txt',  dtype='str')
loc_doc, vocabulary=pos_in_voc(documents, vocabulary)   
phi=0.2
N_D = len(documents)  # num of docs

N_K = 3  # num of topics
max_iter=1000  # number of gibbs sampling iterations

Z, dt, wt, n_z, n_m, lenV=pre_processing(loc_doc,vocabulary, N_K)

alpha,gamma, Pi, B=initialize(N_D, lenV, N_K, loc_doc)
beta_sum,  theta_sum, wtt=lda_gibbs(loc_doc, max_iter, N_K, lenV, n_z, n_m, alpha, gamma, phi, Z, dt, wt, N_D, B, Pi)    

Beta=beta_sum[0]
Theta=theta_sum[0]

Beta=np.array(beta_sum).mean(axis=0)
print("Beta\n",Beta)

### print Theta
for i in range(1,max_iter):
	for j in range(len(Theta)):
		Theta[j]=Theta[j]+theta_sum[i][j]

for i in range(len(Theta)):
	Theta[i]=Theta[i]/max_iter
print("Theta\n",Theta)

Beta
 [[9.22168941e-05 9.22168941e-05 9.22168941e-05 ... 9.24013279e-05
  9.24013279e-05 9.23091110e-05]
 [9.19878576e-05 9.19878576e-05 9.19878576e-05 ... 2.75227670e-04
  2.75319658e-04 3.67859443e-04]
 [9.21998894e-05 9.21998894e-05 9.21998894e-05 ... 9.27530887e-05
  9.26608888e-05 9.21998894e-05]]
Theta
 [[9.38747186e-04 9.88740457e-01 1.87179598e-03]
 [5.61360974e-03 9.94672073e-01 6.44628230e-03]
 [1.90578941e-03 9.98162708e-01 1.56721112e-03]]


### 6.3 Visualization Example (use gensim)

#### In this part, I want to show the visualization of the LDA rresults, since it is an important part in terms of presenting the machine learning results. I used a few packages to set up the model and draw the plot below.

In [26]:
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim


def lda_corpus(corpus_s):
    texts = []
    tokenizer = RegexpTokenizer(r'\w+')

    for doc in corpus_s:
        for word in doc:
            raw = word.lower()
            tokens = tokenizer.tokenize(raw)
            texts.append(tokens)
    
    dictionary = corpora.Dictionary(texts)
    n_corpus = [dictionary.doc2bow(text) for text in texts]
    corpora.MmCorpus.serialize('sample.mm', n_corpus)
    sample = gensim.corpora.MmCorpus('sample.mm')
    
    return sample, dictionary
documents = get_doc('data/sample1.txt',  'data/stopwords.txt')
sample, dic = lda_corpus(documents)

lda_model = gensim.models.ldamodel.LdaModel(corpus = sample,
                                          id2word = dic,
                                          num_topics = 3,
                                          update_every = 1,
                                          chunksize = 1,
                                          passes = 1,
                                          iterations = 5000)
lda_model.print_topics()

C:\Users\Mingjie\AppData\Roaming\Python\Python36\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



[(0,
  '0.213*"noriega" + 0.078*"general" + 0.078*"today" + 0.069*"forces" + 0.069*"call" + 0.069*"defense" + 0.068*"renewed" + 0.037*"officials" + 0.032*"washington" + 0.021*"official"'),
 (1,
  '0.197*"jackson" + 0.095*"public" + 0.087*"economic" + 0.070*"best" + 0.054*"noriegas" + 0.041*"political" + 0.036*"law" + 0.028*"american" + 0.025*"blandon" + 0.025*"drug"'),
 (2,
  '0.147*"letter" + 0.126*"panamanian" + 0.108*"leave" + 0.103*"panama" + 0.088*"people" + 0.038*"monday" + 0.022*"administration" + 0.022*"sanctions" + 0.012*"saying" + 0.011*"central"')]

#### Visualize the topics

In [27]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, sample, dic)
pyLDAvis.display(vis)

## 7. Comparative analysis with competing algorihtms

In this section, I compare LDA model with another two models: LSA and PCA. 

1) LSA (Latent Semantic Analysis):

LSA also known as LSI (Latent Semantic Index), is one of the foundational techniques in topic modeling.  LSA uses bag of word(BoW) model, which results in a term-document matrix(occurrence of terms in a document). Rows represent terms and columns represent documents. LSA learns latent topics by performing a matrix decomposition on the document-term matrix using Singular value decomposition. LSA is typically used as a dimension reduction or noise reducing technique.

2) PCA (Principal component analysis):

We can picture PCA as a technique that finds the directions of maximal variance. PCA finds the projection direction v such that the variance of projected data is maximized. Intuitively, it finds the intrinsic subspace of the original feature space (in terms of retaining the data variability).

#### Define functions to pre-process data

In [4]:
documents = get_doc('data/sample1.txt',  'data/stopwords.txt')
# Vectorize text
vectorizedText = countVectorizer.fit_transform(documents[1]) 
nTopics = 5

In [5]:
def model_fit(Method):
    '''function to get keys categories and counts'''
    if Method == "lda":
        Model = LatentDirichletAllocation(n_components=nTopics, learning_method='online', random_state=0, verbose=0)
    elif Method == "lsi":
        Model = TruncatedSVD(n_components=nTopics)
    else:
        Model = PCA(n_components=nTopics, random_state=0)
    TopicMatrix = Model.fit_transform(vectorizedText.toarray())
    # Get most probable keys and all categories with counts
    Keys = TopicMatrix.argmax(axis=1)
    Categories, Counts = zip(*Counter(Keys).items())
    return Keys, Categories, Counts

In [6]:
def TopWord(n, Keyk, Text, ver):
    '''function to calculate the most frequent words'''
    Mean = np.zeros((nTopics, Text.shape[1]))
    for i in np.unique(Keyk):
        Mean[i] += Text.toarray()[Keyk==i].mean(axis=0)
        
    Word_Indx = np.flip(np.argsort(Mean, axis=1)[:, -n:], axis=1)
    Wor_per = (np.divide(np.flip(np.sort(Mean, axis=1)[:, -n:], axis=1), (np.sum(Mean, axis=1)+0.0000001)[:, None])*100).astype(int)
    topWords = []
    for i, (topic, percentage) in enumerate(zip(Word_Indx, Wor_per)):
        topicWords = []
        if i in np.unique(Keyk):
            for index, percent in zip(topic, percentage):
                wordVector = np.zeros((Text.shape[1]))
                wordVector[index] = 1
                word = ver.inverse_transform(wordVector)[0][0]
                topicWords.append('{}% '.format(percent) + word.encode('ascii').decode('utf-8'))
        # Store all words for the topic
        topWords.append(', '.join(topicWords))

    return topWords

In [30]:
def plotTopics(subject, Categories, Counts, Keys):
    '''function to plot topics'''
    # Sort data
    a, b = zip(*sorted(zip(Categories, Counts)))

    # Create labels
    topWords = TopWord(5, Keys, vectorizedText, countVectorizer)
    labels = ['Topic {}'.format(i) for i in a]
    n = nTopics
    colorm = get_cmap('inferno')
    colors = [rgb2hex(colorm(color)) for color in np.arange(0, 1.000001, 1/(n-1))]

    data = go.Bar(x = labels,
                  y = b,
                  text = [word for word in topWords if word],
                  marker = dict(color = colors))

    layout = go.Layout(title = 'Topics with count'.format(subject),
                       xaxis = dict(title = 'Topic'),
                       yaxis = dict(title = 'Count'),autosize=False,width=800,height=800,)

    fig = go.Figure(data=[data], layout=layout)
    iplot(fig)
    return None

In [15]:
def print_words(Keys):
    '''function to print the most frequent words for each topic'''
    # Get top n words
    topWords = TopWord(5, Keys, vectorizedText, countVectorizer)

    # Print the topics and its words
    for i, words in enumerate(topWords):
        print('Topic {}: {}'.format(i, words))
    return None

#### Method 1: Latent Dirichlet Allocation (lda)

In [16]:
%timeit model_fit("lda")

94.9 ms ± 9.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
ldaKeys, ldaCategories, ldaCounts = model_fit("lda")
print_words(ldaKeys)

Topic 0: 19% oil, 9% san, 6% attorney, 6% israels, 6% bechtels
Topic 1: 19% rappaport, 17% israel, 11% company, 11% foreign, 5% yes
Topic 2: 27% bechtel, 19% memo, 11% meese, 7% years, 7% swiss
Topic 3: 37% peres, 22% official, 11% made, 8% million, 8% ministry
Topic 4: 40% offer, 22% pipeline, 9% francisco, 9% wednesday, 9% sales


In [31]:
plotTopics("LDA",ldaCategories, ldaCounts, ldaKeys)

#### Method 2: Latent Semantic Indexing (LSI)

In [19]:
%timeit model_fit("lsi")

2.15 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
lsiKeys, lsiCategories, lsiCounts = model_fit("lsi")
print_words(lsiKeys)

Topic 0: 61% peres, 28% israel, 9% party, 0% yes, 0% group
Topic 1: 69% offer, 15% near, 15% israeli, 0% yes, 0% group
Topic 2: 20% official, 12% pipeline, 12% memo, 10% made, 7% san
Topic 3: 24% rappaport, 20% oil, 6% yes, 6% wednesday, 6% bechtels
Topic 4: 15% bechtel, 8% company, 8% foreign, 6% million, 6% meese


In [32]:
plotTopics("LSI", lsiCategories, lsiCounts, lsiKeys)

#### Method 3: Principal Component Analysis (PCA)

In [22]:
%timeit model_fit("pca")

1.83 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
pcaKeys, pcaCategories, pcaCounts = model_fit("pca")
print_words(pcaKeys)

Topic 0: 99% peres, 0% yes, 0% group, 0% made, 0% jordan
Topic 1: 99% offer, 0% yes, 0% group, 0% made, 0% jordan
Topic 2: 99% official, 0% yes, 0% group, 0% made, 0% jordan
Topic 3: 6% bechtel, 5% israel, 5% oil, 4% memo, 4% pipeline
Topic 4: 99% rappaport, 0% friend, 0% made, 0% jordan, 0% israels


In [33]:
plotTopics("PCA", pcaCategories, pcaCounts, pcaKeys)

Based on the results above, we can see that the fastest model is PCA and the slowest is LDA. Also we can see that the results given by PCA are very different from that given by LDA and LSI.

Topic Modeling automatically discover the hidden themes from given documents. 
LSI algorithm is a simple method which is easy to understand and implement. It also offers better results compared to the vector space model. LSA uses SVD, and as a result the topics are assumed to be orthogonal. It's much faster than LDA, but LDA prevents over-fitting, and gives better results because it uses dirichlet priors for the document-topic and topic-word distributions. 

As an unsupervised learning algorithm, PCA provides uneven distributed counts across the 5 topics, as shown in the plot above. And the topic words of first two topics given by PCA are mostly the same--making it difficult to trust the accuracy of PCA model. We may conclude that PCA is not optimal for this kind of classification. Note that there is no mention of the class label in the definition of PCA, keeping the dimensions of largest energy (variance) is a good idea but not always enough. PCA certainly improves the density estimation, since space has smaller dimension, but could be unwise from a classification point of view. 

## 8. Discussion/conclusion

One of the primary applications of natural language processing is to automatically extract what topics people are discussing from large volumes of text. Some examples of large text could be feeds from social media, customer reviews of hotels, movies, etc, user feedbacks, news stories, e-mails of customer complaints etc.

In this report, I successfully implement the LDA model with python. To optimize the algorithm, I applied variational EM, JIT, timeit and cython. I tested the algorithm using generated test data set, and applied the algorithm to real data set for topic modeling. In addition, I compared LDA model with other types of models: LSI and PCA. 

LDA’s approach to topic modeling is to consider each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion. Once you provide the algorithm with the number of topics, all it does is to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

LDA is easily the most popular (and typically most effective) topic modeling technique out there. With LDA, we can extract human-interpretable topics from a document corpus, where each topic is characterized by the words they are most strongly associated with. These vectors are often very useful for downstream applications. In addition to topic modeling, LDA could also be generalized to other problem domains, such as clustering images and image classification.

LDA also has some limitations. First, it is very difficult to analyze the results and it is tough to draw insights from the outputs, because the outcomes are mostly random words. We may need some domain expert for insights and decide whether the results are better suited for a particular case. Second, LDA models could be very sensitive to the input data. With small changes to the stemming/tokenisation process, the output topics can be completely different. In addition, topics could be "unstable" in the sense that adding new documents can cause significant changes to the topic distribution.

## 9. References

1) D.M. Blei, A.Y. Ng, and M.I. Jordan. Latent Dirichlet allocation. Journal of Machine Learning Research, 3:993–1022, 2003.

2) D.M. Blei and J. Lafferty. Topicmodels. Text Mining: Theory and Applications, 2009